<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library
!pip install -qU magenta
import magenta
import note_seq
import tensorflow
from note_seq.protobuf import music_pb2
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from IPython.display import clear_output
from IPython.display import Audio
clear_output()

In [3]:
# Создаем свою последовательность с помощью note_seq

teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=80)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

note_seq.plot_sequence(teapot)
note_seq.play_sequence(teapot,synth=note_seq.synthesize)

In [4]:
# Создаем midi файл с именем sample_output.mid, содержащий звуки teapot
note_seq.sequence_proto_to_midi_file(teapot, 'sample_output.mid')

### Создайте композицию music_pb2.NoteSequence(), после чего в MelodyRNN загрузите вместо “basic_rnn.mag” следующие bundle: “mono_rnn.mag”, “lookback_rnn.mag” и “attention_rnn.mag”. Для каждого из них продлите созданную Вами композицию

In [8]:
# =========
# MelodyRNN
# =========
# https://arxiv.org/pdf/1712.05274.pdf


# Загружаем basic_rnn.mag. Так же можно загружать любые другие модели от Magenta

note_seq.notebook_utils.download_bundle('mono_rnn.mag', '/content/')

bundle = sequence_generator_bundle.read_bundle_file('/content/mono_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['mono_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

INFO:tensorflow:Restoring parameters from /tmp/tmpv_s6umb4/model.ckpt


In [9]:
# Можем настроить количество шагов в новой последовательности
# Также «температуру» результата - чем выше температура, тем более случайной (и менее похожей на ввод) будет наша последовательность.


input_sequence = teapot # Последовательность
num_steps = 128 # Длина новой последовательности
temperature = 1.0 # Температура


last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
    start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)

# Сгенерируем последовательность по параметрам которые объявили выше.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -75.386650 


In [11]:
note_seq.notebook_utils.download_bundle('lookback_rnn.mag', '/content/')

bundle = sequence_generator_bundle.read_bundle_file('/content/lookback_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['lookback_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpyl2fkcbm/model.ckpt


In [12]:
# Можем настроить количество шагов в новой последовательности
# Также «температуру» результата - чем выше температура, тем более случайной (и менее похожей на ввод) будет наша последовательность.


input_sequence = teapot # Последовательность
num_steps = 128 # Длина новой последовательности
temperature = 1.0 # Температура


last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
    start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)

# Сгенерируем последовательность по параметрам которые объявили выше.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -88.689415 


In [13]:

note_seq.notebook_utils.download_bundle('attention_rnn.mag', '/content/')

bundle = sequence_generator_bundle.read_bundle_file('/content/attention_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['attention_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpzc_c96kk/model.ckpt


In [14]:
# Можем настроить количество шагов в новой последовательности
# Также «температуру» результата - чем выше температура, тем более случайной (и менее похожей на ввод) будет наша последовательность.


input_sequence = teapot # Последовательность
num_steps = 128 # Длина новой последовательности
temperature = 1.0 # Температура


last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
    start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)

# Сгенерируем последовательность по параметрам которые объявили выше.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -108.249664 


последний вариант поприятнее будет, хотя и предыдущие неплохо. Стиль сохранен